## EVAL1: evaluation of likelihood function and its derivatives. It differs from EVAL by the inclusion of the lagged dependent variable i(t-1)

In [60]:
import numpy as np
import cmath

In [531]:
ev = np.array([[1,3,5],[7,9,11],[13,15,17]])
c = np.array([[2,4,6],[8,10,12],[14,16,18]])
bet = 0.6
d1v = np.array([[-1,2,4],[-1,8,10],[-1,14,16]])
dll = np.array([[1],[2],[3]])
lp = np.array([[2],[4],[6]])
pk = np.array([[3,5,7],[9,11,13],[15,17,19]])
dt = np.array([[3,1,1,1],[4,2,2,2],[5,3,3,3]])
q = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
tr = np.array([[3,5,7],[9,11,13],[15,17,19]])
lnp = np.array([[2,4,6],[8,10,12],[14,16,18],[20,22,24],[26,28,30]])
dev = np.array([[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15]])
dm = 4
g = 1
modnum = np.array([[1,2,1,0,4]])
dl = np.array([[2,4,6,8,10],[12,1,2,3,14],[16,18,20,22,24]])
dc = np.array([[2,4,6],[8,10,12],[14,16,18]])
invp = np.array([[1,2,3],[5,6,7],[9,10,11],[13,14,15],[17,18,19]])
nt = 3

In [3]:
def eval1_():
    
    #EVALUATE PROBABILITY OF KEEPING BUS ENGINE AT EACH SAMPLE POINT

    if bet > 0:
        lp1 = (1/(1+np.exp(c-bet*ev-tr-c[0,0]+bet*ev[0,0]+q[(dm-1),0])))[dt[:,1]-1,:]
        
    else:
        lp1 = (1/(1+np.exp(c-tr-c[0,0]+q[dm-1,0])))[dt[:,1]-1,:]
        
    lp = pk[dt[:,1]-1,:]
    
    if i < 0:
        #Go to loglik
        print("Go to loglik")
        
        
    #COMPUTE DERIVATIVES OF LOGLIKELIHOOD FUNCTION
    
    
    #CASE 1: ESTIMATE FULL LIKELIHOOD BUT NOT bet
    
    if (modnum[0,2:4].flatten() == np.array([[2,0]])).all():
        
        d1v = np.concatenate([-np.ones((nt,1)), dc[dt[:,1]-1,:], np.zeros((nt, modnum[0,4]))], axis=1)
    
    
    #CASE 2: ESTIMATE FULL LIKELIHOOD AND bet
        
    elif (modnum[0,2:4].flatten() == np.array([[2,1]])).all():
        
        d1v = np.concatenate([(-np.ones((nt,1))), (dc[dt[:,1]-1,:]), (-bet*(1-bet)*(ev[0,0]-ev[dt[:,1]-1])), np.zeros((nt, modnum[0,4]))], axis=1)
        
   
    #CASE 3: ESTIMATE NEITHER PARTIAL LIKELIHOOD NOR bet
    
    elif (modnum[0,2:4].flatten() == np.array([[1,0]])).all():
        
        d1v = np.concatenate([(-np.ones((nt,1))), dc[dt[:,1]-1,:],], axis=1)
        
    
    #CASE 4: ESTIMATE PARTIAL LIKELIHOOD AND bet
    
    elif (modnum[0,2:4].flatten() == np.array([[1,1]])).all():
        
        d1v = np.concatenate([(-np.ones((nt,1))), (dc[dt[:,1]-1,:]),(-bet*(1-bet)*(ev[0,0]-ev[dt[:,1]-1,:]))], axis=1 )                 
        
    

## loglik

In [302]:
    #CUMULATE LOGLIKELIHOOD FUNCTION

    
    #FULL LOGLIKELIHOOD FUNCTION

    if modnum[0,2] == 2:
        
        ll = ll + np.array([np.sum(np.vectorize(cmath.log)(((lp+(1-2*lp)*np.array([dt[:,0]]).T)*np.array([dt[:,3]]).T+(lp1+(1-2*lp1)*np.array([dt[:,0]]).T)*(1-np.array([dt[:,3]]).T)))+ lnp[dt[:,2],:], axis=0)]).T

    
    #PARTIAL LOGLIKELIHOOD FUNCTION
    
    else:
        
        ll = ll + np.array([np.sum(np.vectorize(cmath.log)(((lp+(1-2*lp)*np.array([dt[:,0]]).T)*np.array([dt[:,3]]).T+(lp1+(1-2*lp1)*np.array([dt[:,0]]).T)*(1-np.array([dt[:,3]]).T))), axis=0)]).T
        
    
    if i < 0:
        
        #Go to bypass
        print("Go to bypass")
        
    lp1 = ((lp-1+np.array([dt[:,0]]).T)*np.array([dt[:,3]]).T+(lp1-1+np.array([dt[:,0]]).T)*np.array([1-dt[:,3]]).T) 
    
    
    #ADD ON DERIVATIVES OF EXPECTED VALUE FUNCTION, BYPASS IF bet=0
    
    if bet == 0:
        
        d1v = d1v * lp1
    
        if modnum[0,2] == 2:
            
            g = dm - modnum[0,4]
            d1v[:,g-1:dm-1] = invp[dt[:,2],:]
            
            
        #Go to cumulate
        print("Go to cumulate")
            

Go to bypass


In [345]:
    #FIRST TWO DERIVATIVES (tr,c1) ARE SAME FOR ALL MODELS

#In Gauss, I removed a column from lp1 to make sure the matrices are comfortable and then checking if the outputs are the same.

    d1v[:,0:2]=(d1v[:,0:2]+dev[0,0:2] * np.ones((nt,1))-dev[:,0:2][dt[:,1]-1,:])*lp1


    #DERIVATIVES FOR SUMMANDS OF PARTIAL LOGLIKELIHOOD FUNCTION
    
    if modnum[0,2] == 1:
        
        for g in range(3, dm):
            u = (((np.array([d1v[:,(g-1)]]).T) + (np.array([dev[0,(g-1)]]).T)*np.ones((nt,1)))-(np.array([dev[:,(g-1)]]).T)[dt[:,1]-1,:])*lp1
            d1v[:,(g-1)] = u.flatten()
            
    elif modnum[0,2] == 2:
        
        for g in range(3,dm-modnum[0,4]):
            u = (d1v[:,(g-1)]+ dev[0,(g-1)]*np.ones(nt,1)-dev[:,(g-1)][dt[:,1]-1,:])*lp1
            d1v[:,(g-1)] = u.flatten()
        
        
        g = dm-int(modnum[0,4])
        d1v[:,(g-1):(dm-1)]=(dev[0,(g-1):(dm-1)]*np.ones((nt,1))-dev[:,(g-1):dm-1][dt[:,1]-1,:])*lp1+invp[dt[:,2],:]


## CUMULATE

In [ ]:
#CUMULATE DERIVATIVES OF LOGLIKELIHOOD FUNCTION

d1v = np.concatenate([d1v, ((1-np.array([dt[:,3]])).T*lp1)], axis=1)

dll = dll + np.array([np.sum(d1v, axis=0)]).T

## BYPASS

In [537]:
print("bypass:")

#end of the procedure

bypass:
